# Blurry licence plates

In [1]:
from pathlib import Path
import cv2
import numpy as np
import pytesseract

In [2]:
w_dir = "licence_plates/blurry/"
files = [
    'incorrect_8EVI580_8FVI580.jpg',
    'incorrect_8F72061_8F72041.jpg',
    'incorrect_8FDD500_BFDD500.jpg',
    'incorrect_8JYF020_8JVF020.jpg',
    'incorrect_8MW8969_8MWB969.jpg']

### Make a template

In [19]:
frame = cv2.imread(w_dir + files[2])
h, w, _ = frame.shape
print(f"w = {w}, h = {h}")

template = {
    'l1': {'x1': 7, 'y1': 20, 'w': 18, 'h': 32},
    'l2': {'x1': 23, 'y1': 20, 'w': 18, 'h': 32},
    'l3': {'x1': 39, 'y1': 20, 'w': 18, 'h': 32},
    'l4': {'x1': 55, 'y1': 20, 'w': 18, 'h': 32},
    'l5': {'x1': 71, 'y1': 20, 'w': 18, 'h': 32},
    'l6': {'x1': 87, 'y1': 20, 'w': 18, 'h': 32},
    'l7': {'x1': 103, 'y1': 20, 'w': 18, 'h': 32},
}

color = (0,255,0)
for t in template.values():
    cv2.rectangle(frame, (t['x1'], t['y1']), (t['x1'] + t['w'], t['y1'] + t['h']), color, 1)

cv2.imwrite(w_dir + "tmp.jpg", frame)

w = 128, h = 64


True

In [17]:
frame[:,:,:] = 0
color = (0,255,0)
for t in template.values():
    cv2.rectangle(frame, (t['x1'], t['y1']), (t['x1'] + t['w'], t['y1'] + t['h']), color, 1)

cv2.imwrite(w_dir + "template.jpg", frame)

True

### OCR of single symbols (words)

In [14]:
# Using psm 8 because we only have a single “word”
pytesseract.pytesseract.tesseract_cmd = 'tesseract'
custom_oem_psm_config = '--dpi 300 --oem 3 --psm 8'

def prepare_symbol(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    gray = cv2.medianBlur(gray, 3)
    return gray

def ocr_symbol(img):
    gray = prepare_symbol(img)
    return pytesseract.image_to_string(gray, config=custom_oem_psm_config, timeout=2).strip()

def detect_lp(img, template):
    plate = ocr_symbol(img)
    symbols = []
    for t in template.values():
        smbl_img = img[t['y1']: t['y1'] + t['h'], t['x1']: t['x1'] + t['w']]
        symbols.append(ocr_symbol(smbl_img))
    return plate, ''.join(symbols)

In [15]:
for f in files:
    img = cv2.imread(w_dir + f)
    plate, symbols = detect_lp(img, template)
    print(f"{f} => {plate} ~ {symbols}")

incorrect_8EVI580_8FVI580.jpg => LaF VTSAQ) ~ 8F¥I5a0
incorrect_8F72061_8F72041.jpg => BF 72C41 ~ ar7?1,I
incorrect_8FDD500_BFDD500.jpg => prnpson§ ~ QF7fn5fi
incorrect_8JYF020_8JVF020.jpg => | ~ -74c93.
incorrect_8MW8969_8MWB969.jpg => fevszse9] ~ if¥‘Yd19i9


In [6]:
for f in files:
    name = Path(f).stem
    out_dir = Path(w_dir).joinpath(name)
    out_dir.mkdir(parents=True, exist_ok=True)
    img = cv2.imread(str(Path(w_dir).joinpath(f)))
    for l, t in template.items():
        smbl_img = img[t['y1']: t['y1'] + t['h'], t['x1']: t['x1'] + t['w']]
        cv2.imwrite(str(out_dir.joinpath(f"{l}.jpg")), smbl_img)
        gray_img = prepare_symbol(smbl_img)
        cv2.imwrite(str(out_dir.joinpath(f"{l}_.jpg")), gray_img)

In [11]:
def prepare_symbol(img):
#     lab= cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
#     l, a, b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
#     cl = clahe.apply(l)
#     limg = cv2.merge((cl,a,b))
#     img = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    gray = cv2.medianBlur(gray, 3)
    return gray

dir_ = Path(w_dir).joinpath("incorrect_8EVI580_8FVI580")
for n in range(1,8):
    smbl_img = cv2.imread(str(dir_.joinpath(f"l{str(n)}.jpg")))
    gray = prepare_symbol(smbl_img)
    cv2.imwrite(str(dir_.joinpath(f"l{str(n)}_.jpg")), gray)
    c = pytesseract.image_to_string(gray, config=custom_oem_psm_config, timeout=2).strip()
    print(c)

8
F
¥
I
5
8
0


In [ ]:
8F¥I5a0